<a href="https://colab.research.google.com/github/datajcthemax/playdata/blob/main/day6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   MySQL 데이터베이스 설치
*   Database에 대하여


*   Select등 구문 정리
*   Lotto API 수집 내용 DB 넣기 실습
*   병원관리 프로그램에 사용할 데이터 구조 구상





mysql에서 select 실습

In [ ]:
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 51.3 MB/s eta 0:00:00


In [ ]:
from mysql import connector

In [ ]:
conn = connector.connect(host= '52.78.193.207', user='myname' , password='1234' , database='mydb', port='53339' )

In [ ]:
conn

In [ ]:
cursor = conn.cursor()
cursor.execute('select * from lotto_numbers;')

In [ ]:
rows = cursor.fetchall()

In [ ]:
rows

[(1, 1, 1, 1, 1, 1, 1, 1, 1)]

실습
1. 위에 내용을 토대로 mysql에 접속해서 조회하는 함수를 만들어 봅시다.
2. 클래스에도 적용해보자

In [ ]:
#조회, 입력, 수정, 삭제
#-지난시간에 했던 로또 번호 수집기능과 결합해서
#-DB에 입력하는 프로그램을 만들어 봅시다(입력)
#-입력된 데이터를 확인하는 기능(조회)
#-수정하는 기능(수정)
#-삭제하는 기능
#을 넣은 클래스를 만들어 봅시다.

In [ ]:
import requests
import mysql.connector

class Lotto:
    def __init__(self, max_round, host, user, password, database, port):        
        self.max_round = max_round
        # 연결하기
        self.connection = mysql.connector.connect(
            host = host,
            user = user,
            password = password,
            database = database,
            port = port
        )
        
    def get_api(self):
      datas = []
      max_round = self.max_round
      for round in range(1, max_round+1):
        response = requests.get(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={round}")
        data = response.json()
        datas.append(data)
      return datas

    def lot_data(self):
      datas = self.get_api()      
      wanted_keys = ["drwNo", "drwtNo1", "drwtNo2", 'drwtNo3', 'drwtNo4', 'drwtNo5', 'drwtNo6', 'bnusNo']
      for data in datas:
        tmp_data = [data[key] for key in wanted_keys]
        sql = "INSERT into lotto_numbers_im (roundNum, num1,num2, num3,num4, num5, num6, bonus) values({},{},{},{},{},{},{},{});".format(*tmp_data)
        self.execute_query(sql)      
   
    def execute_query(self, query): 
      cursor = self.connection. cursor()
      cursor.execute(query)
      self.connection.commit()
      # result = cursor.fetchall()         

    def select(self):
      cursor = self.connection. cursor()
      query="select * from lotto_numbers_im"
      cursor.execute(query)
      rows = cursor.fetchall()
      print(rows)
    def close(self):
      self.connection.close()

In [ ]:
# 연결 하기
# api통해서 조회 - 모든 회차를 받기
# 받은 내용을 DB 입력
# 입력된 내용 확인
# 연결 끊기

HOST = "13.124.204.183"
PORT = "51119"
USER = "myname"
PW = "1234"
DATABASE = 'mydb'
lotto = Lotto(max_round=3, host=HOST, user=USER, password=PW, database=DATABASE, port=PORT)